# Import Toronto Neighborhoods data from Wikipedia and Clean it

In [1]:
#libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#request data from wikipedia website
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url)

In [5]:
#create a dataframe with the data
df_raw = pd.read_html(source.content, header=0)[0]

#Exclude "Not Assigned"rows
df = df_raw[df_raw.Borough != "Not assigned"]

#Assign empty Neighborhoods with Borough names
df.Neighborhood.replace('Not assigned',df.Borough,inplace=True)

In [11]:
#Combine same Borough Neighboorhoods
df_toronto = df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x))
df_toronto = df_toronto.reset_index()
df_toronto.rename(columns = {'Postcode':'PostalCode'}, inplace = True)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df_toronto.shape

(103, 3)